In [67]:
#pip install -q -U keras-tuner

In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random 

In [69]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_squared_error, r2_score
import keras_tuner as kt

In [70]:
tf.random.set_seed(100)

In [71]:
df = pd.read_csv("datasets\covid19postvaxstatewidestats.csv")

df = df[df['unvaccinated_cases_per_100k'].notna()]

df = df.reset_index(drop=True)

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\dangn\AppData\Local\Temp\ipykernel_7324\2108600221.py:1: SyntaxWarning: invalid escape sequence '\c'
  df = pd.read_csv("datasets\covid19postvaxstatewidestats.csv")


In [72]:
df_unvac = df[['unvaccinated_cases_per_100k', 'unvaccinated_hosp_per_100k', 'unvaccinated_deaths_per_100k']]

df_unvac

,unvaccinated_cases_per_100k,unvaccinated_hosp_per_100k,unvaccinated_deaths_per_100k
0,27.251295,1.483401,0.067702
1,25.072362,1.360519,0.078033
2,23.268146,1.296892,0.087677
3,21.766329,1.236441,0.099877
4,20.467271,1.179528,0.112219
...,...,...,...
778,8.425650,0.613146,0.051096
779,8.210084,0.564715,0.048550
780,8.268116,0.552060,0.048561
781,8.157356,0.570069,0.061353


In [73]:
def sequence_data(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), :])
        y.append(data[i+time_step,:])
    return np.array(X), np.array(y)

In [74]:
train_data = df_unvac.values

In [75]:
X_train, y_train = sequence_data(train_data, 10)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])

In [76]:
X_train.shape

(772, 10, 3)

In [77]:
y_train.shape

(772, 3)

In [78]:
def model_builder(hp):
    model = Sequential()
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)

    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(LSTM(units = hp_layer_1, activation=hp_activation, input_shape = (10,3)))

    model.add(Dense(units = hp_layer_2, activation = hp_activation))

    model.add(Dense(units=3, activation=hp_activation))


    model.compile(optimizer = Adam(learning_rate=hp_learning_rate), loss='mse', metrics=['mae'])

    return model

In [79]:
tuner = kt.Hyperband(model_builder, 
                     objective='val_mae',
                     max_epochs=10,
                     factor = 3,
                     directory = 'dir',
                     project_name = 'one')

C:\Users\dangn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [80]:
stop_early = EarlyStopping(monitor = 'val_mae', patience = 3)

In [81]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 03s]
val_mae: 1.2989795207977295

Best val_mae So Far: 0.6418715715408325
Total elapsed time: 00h 01m 30s


In [82]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

In [83]:
print("The best hyperparameters are:")
for key, value in best_hp.values.items():
    print(f"{key}: {value}")

The best hyperparameters are:
activation: relu
layer_1: 448
layer_2: 288
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0014


In [84]:
model = tuner.hypermodel.build(best_hp)

In [85]:
history = model.fit(X_train, y_train, epochs = 50, validation_split = 0.2, callbacks = [stop_early])

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 4674.0498 - mae: 20.2616 - val_loss: 221.1450 - val_mae: 7.8075
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 4667.4604 - mae: 19.9560 - val_loss: 221.1325 - val_mae: 7.7748
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 4667.6182 - mae: 19.9582 - val_loss: 221.1269 - val_mae: 7.7635
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4667.1533 - mae: 19.9008 - val_loss: 221.1254 - val_mae: 7.7556
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4666.9360 - mae: 19.8532 - val_loss: 221.1319 - val_mae: 7.7726
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4667.0269 - mae: 19.8716 - val_loss: 221.1210 - val_mae: 7.7455
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4667.2622 - mae: 19.8616 - val_loss: 221.1216 - val_mae: 7.7493
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 4666.9551 - mae: 19.8392 - val_loss: 221.1312 - val_mae: 7.7674
Epoch 9/50
20/20 ━━━━━━

In [86]:
pd.DataFrame(history.history)

,loss,mae,val_loss,val_mae
0,4496.041016,19.955139,221.145004,7.807496
1,4493.800781,19.799435,221.132507,7.774765
2,4494.010742,19.810051,221.126923,7.763500
3,4493.457520,19.741285,221.125397,7.755567
4,4493.357422,19.709314,221.131912,7.772570
5,4493.368652,19.713688,221.120987,7.745506
6,4493.550293,19.711470,221.121643,7.749313
7,4493.365723,19.693708,221.131195,7.767393
8,4493.582031,19.711222,221.136917,7.782497


In [87]:
def model_builder(hp):
    model = Sequential()
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)

    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)

    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(LSTM(units = hp_layer_1, activation=hp_activation, input_shape = (10,3)))

    model.add(Dense(units = hp_layer_2, activation = hp_activation))

    model.add(Dense(units = hp_layer_3, activation = hp_activation))
    
    model.add(Dense(units=3, activation=hp_activation))


    model.compile(optimizer = Adam(learning_rate=hp_learning_rate), loss='mse', metrics=['mae'])

    return model

In [88]:
tuner = kt.Hyperband(model_builder, 
                     objective='val_mae',
                     max_epochs=10,
                     factor = 3,
                     directory = 'dir',
                     project_name = 'second')

C:\Users\dangn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [89]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 05s]
val_mae: 0.9176017642021179

Best val_mae So Far: 0.49750617146492004
Total elapsed time: 00h 01m 47s


In [90]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

In [91]:
print("The best hyperparameters are:")
for key, value in best_hp.values.items():
    print(f"{key}: {value}")

The best hyperparameters are:
activation: relu
layer_1: 320
layer_2: 448
layer_3: 416
learning_rate: 0.0001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012


In [ ]:
model_1 = tuner.hypermodel.build(best_hp)